In [1]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from funpymodeling.exploratory import freq_tbl
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('Mexico.csv')

In [3]:
df= df.fillna(method="bfill")
df= df.fillna(method="ffill")
df = df.bfill()
df= df.ffill()

C:\Users\HP\AppData\Local\Temp\ipykernel_19992\1730909967.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method="bfill")
C:\Users\HP\AppData\Local\Temp\ipykernel_19992\1730909967.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method="ffill")


**Transformando variables a tipo dicotómicas**

In [4]:

#bathrooms_text
df["bathroomsd"] = df["bathrooms_text"].astype(str).apply(lambda x: 1 if "2" in x or "3" in x or "4" in x else 0)
#room_type Entire home
df["E_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Entire home/apt" else 0)
#room_type Private home
df["P_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Private room" else 0)
#room_type Hotel home
df["H_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Hotel room" else 0)
#instant_bookabled
df["instant_bookabled"] = df["instant_bookable"].apply(lambda x: 1 if x == "t" else 0)
#Price
df["price"] = df["price"].replace('[\$,]', '', regex=True).astype(float)
Mprice = df["price"].median()
df["D_price"] = df["price"].apply(lambda x: 1 if x >= Mprice else 0)
#host_total_listings_count
Mlistings = df["host_total_listings_count"].median()
df["D_host_listings"] = df["host_total_listings_count"].apply(lambda x: 1 if x >= Mlistings else 0)
#host_total_listings_count
Maccuracy = df["host_total_listings_count"].median()
df["D_accuracy"] = df["review_scores_accuracy"].apply(lambda x: 1 if x >= Maccuracy else 0)
#availability_30
Mavailability = df["availability_30"].median()
df["D_availability"] = df["availability_30"].apply(lambda x: 1 if x >= Mavailability else 0)
#availability_90
Mavailability90 = df["availability_90"].median()
df["D_availability90"] = df["availability_90"].apply(lambda x: 1 if x >= Mavailability90 else 0)
#number_of_reviews
Mreviews = df["number_of_reviews"].median()
df["D_reviews"] = df["number_of_reviews"].apply(lambda x: 1 if x >= Mreviews else 0)
#review_scores_rating
Mrating = df["review_scores_rating"].median()
df["D_rating"] = df["review_scores_rating"].apply(lambda x: 1 if x >= Mrating else 0)
#review_scores_rating
Maccommodates = df["accommodates"].median()
df["D_accommodates"] = df["accommodates"].apply(lambda x: 1 if x >= Maccommodates else 0)



<>:12: SyntaxWarning: invalid escape sequence '\$'
<>:12: SyntaxWarning: invalid escape sequence '\$'
C:\Users\HP\AppData\Local\Temp\ipykernel_19992\2275433636.py:12: SyntaxWarning: invalid escape sequence '\$'
  df["price"] = df["price"].replace('[\$,]', '', regex=True).astype(float)


In [6]:
df.head(5)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,H_room_typed,instant_bookabled,D_price,D_host_listings,D_accuracy,D_availability,D_availability90,D_reviews,D_rating,D_accommodates
0,35797,https://www.airbnb.com/rooms/35797,20241227033128,2024-12-27,city scrape,Villa Dante,"Dentro de Villa un estudio de arte con futon, ...","Santa Fe Shopping Mall, Interlomas Park and th...",https://a0.muscache.com/pictures/f395ab78-1185...,153786,...,0,0,1,0,0,1,1,0,0,1
1,44616,https://www.airbnb.com/rooms/44616,20241227033128,2024-12-28,city scrape,Condesa Haus,A new concept of hosting in mexico through a b...,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/251410/ec75fe...,196253,...,0,0,1,1,0,1,1,1,0,1
2,67703,https://www.airbnb.com/rooms/67703,20241227033128,2024-12-28,city scrape,"2 bedroom apt. deco bldg, Condesa","Comfortably furnished, sunny, 2 bedroom apt., ...",Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/3281720/6f078...,334451,...,0,0,1,0,0,1,0,1,1,1
3,70644,https://www.airbnb.com/rooms/70644,20241227033128,2024-12-27,city scrape,Beautiful light Studio Coyoacan- full equipped !,COYOACAN designer studio quiet & safe! well eq...,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,212109,...,0,0,1,0,0,0,0,1,1,1
4,131610,https://www.airbnb.com/rooms/131610,20241227033128,2024-12-28,previous scrape,MARIA DEL ALMA,Welcome to Your Home in Mexico City<br />We ha...,San Miguel Chapultepec is the best kept secret...,https://a0.muscache.com/pictures/837085/b9ed71...,647454,...,0,0,1,0,0,0,0,0,1,1


**CASO 1: accommodates, bathrooms, room_type**

In [7]:
#Declaramos las variables dependientes e independiente para las regresion Logisitica 
Vars_Indep = df[['D_accommodates', 'bathroomsd', 'E_room_typed']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

#Se escalan los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [8]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[1894 2024]
 [ 601 3366]]


In [9]:
#Caulculo de la precisuin del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.6244897959183674


In [10]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6670894102726697


In [11]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.8485001260398286


**CASO 2:** 
**price, host_listings, accuracy**

In [11]:
#Declaramos las variables dependientes e independiente para las regresion Logisitica 
Vars_Indep = df[['D_price', 'D_host_listings', 'D_accuracy']]
Var_Dep = df['D_availability']
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

#Se escalan los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [12]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[ 636 3153]
 [ 624 3471]]


In [13]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5240036231884058


In [14]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5209284627092846


In [15]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.8476190476190476


**CASO 3:** 
**availability, number_of_reviews, review_scores_rating**

In [16]:
#Declaramos las variables dependientes e independiente para las regresion Logisitica 
Vars_Indep = df[['D_availability', 'D_reviews', 'D_rating']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

#Se escalan los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [17]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[2087 1840]
 [1802 2155]]


In [18]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5394242803504381


In [19]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5380517503805176


In [20]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.5446044983573414


**CASO 4:** 
**bathrooms, number_of_reviews, price**

In [21]:
# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['bathroomsd', 'D_reviews', 'D_price']]
Var_Dep = df['D_availability']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([0, 0, 0, ..., 1, 1, 0], dtype=int64)

In [22]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[1545 2276]
 [1379 2684]]


In [23]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5411290322580645


In [24]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5364028411973617


In [25]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.6605956190007384


**CASO 5:** 
**accommodates, review_scores_rating, availability_30**

In [26]:
# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['D_accommodates', 'D_reviews', 'D_availability']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

In [27]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[ 753 3122]
 [ 116 3893]]


In [28]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5549536707056308


In [29]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5892947742262811


In [30]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.9710651035170865


**CASO 6:** 
**instant_bookable, review_scores_rating, availability_90**

In [31]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['instant_bookabled', 'D_reviews', 'D_availability90']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 0, 0, ..., 1, 0, 1], dtype=int64)

In [32]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[1980 1928]
 [1925 2051]]


In [33]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5154561447599899


In [34]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5112886859462202


In [35]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.5158450704225352


**CASO 7:** 
**bathrooms, availability_30, host_listings**

In [36]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['bathroomsd', 'D_availability', 'D_host_listings']]
Var_Dep = df['D_reviews']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [37]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[1884 1981]
 [1735 2284]]


In [38]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5355216881594372


In [39]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5286656519533232


In [40]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.5683005722816621


**CASO 8:** 
**price, accommodates, review_scores_rating**

In [41]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['D_price', 'D_accommodates', 'D_rating']]
Var_Dep = df['D_reviews']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [42]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
exactitud = accuracy_score(y_test, y_pred)

print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[1742 2166]
 [1300 2676]]


In [43]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5526641883519207


In [44]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5603754439370878


In [45]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.6730382293762576


**CASO 9:** 
**bathrooms, review_scores_accuracy, review_scores_rating**

In [46]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['bathroomsd', 'D_accuracy', 'D_rating']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [47]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
exactitud = accuracy_score(y_test, y_pred)

print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[3391  499]
 [2307 1687]]


In [48]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.7717291857273559


In [49]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6440892947742263


In [50]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.42238357536304455


**CASO 10:** 
**availabilit_90, host_total_listings_count, number_of_reviews**

In [51]:
# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['D_availability90', 'D_host_listings', 'D_reviews']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([0, 1, 0, ..., 0, 1, 1], dtype=int64)

In [52]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
exactitud = accuracy_score(y_test, y_pred)

print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[1942 1980]
 [1902 2060]]


In [53]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5099009900990099


In [54]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5076103500761036


In [55]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.5199394245330641
